In [1]:
#%matplotlib notebook
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
plt.style.use('seaborn-pastel')
import matplotlib.gridspec as gridspec
from ipywidgets import interact, interactive, fixed
from scipy.optimize import fsolve
import nbinteract as nbi


In [2]:
#nbi:hide_in
def J1(a,b,c1,c2,d,h1,h2):
    return a-b*(h1+h2)-b*h1-c1-d*h1

def J2(a,b,c1,c2,d,h1,h2):
    return a-b*(h1+h2)-b*h2-c2-d*h2

def unconstrainedCournot(a,b,c1,c2,d):
    Q=(2*a-c1-c2)/(2*b+b+d)
    q1=(a-b*Q-c1)/(b+d)
    q2=(a-b*Q-c2)/(b+d)
    return [q1,q2]
def BR1(a,b,c1,d,h2):
    return max(0,(a-b*h2-c1)/(2*b+d))
def BR2(a,b,c2,d,h1):
    return max(0,(a-b*h1-c2)/(2*b+d))

def harvestFunction(a,b,c1,c2,d,e1,e2):
    if e1+e2 > sum(unconstrainedCournot(a,b,c1,c2,d)):
        if e1< unconstrainedCournot(a,b,c1,c2,d)[0]:
            return [e1,BR2(a,b,c2,d,e1)]
        elif e2 <unconstrainedCournot(a,b,c1,c2,d)[1]:
            return [BR1(a,b,c1,d,e2),e2]
        else:
            return unconstrainedCournot(a,b,c1,c2,d)
    else:
        if J1(a,b,c1,c2,d,e1,e2)>0 and J2(a,b,c1,c2,d,e1,e2)<0:
            return [e1,BR2(a,b,c2,d,e1)]
        elif J1(a,b,c1,c2,d,e1,e2)<0 and J2(a,b,c1,c2,d,e1,e2)>0:
            return [BR1(a,b,c1,d,e2),e2]
        else:
            return [e1,e2]


def costFcn(c1,c2,d,h1,h2):
    return c1*h1+0.5*d*h1**2 + c2*h2+0.5*d*h2**2

def revenueFcn(a,b,h1,h2):
    return (a-b*(h1+h2))*(h1+h2)

def totalProfit(a,b,c1,c2,d,h1,h2):
    prfit= revenueFcn(a,b,h1,h2) - costFcn(c1,c2,d,h1,h2)
    return prfit

def welfare(a,b,c1,c2,d,h1,h2):
    return (a+a-b*(h1+h2))*(h1+h2)/2-c1*h1-0.5*d*h1**2 - c2*h2-0.5*d*h2**2

def indexSet(l):
    mx=max(l)
    idx=[]
    for i in range(len(l)):
        if l[i]==mx:
            idx.append(i)
    return idx

#for a given share cap s and Qbar, return E*
def optE(a,b,c1,c2,d,Qbar,s):
    e1list=np.linspace(Qbar*s,Qbar*(1-s), int((2*s-1)*1000)+1)
    e2list=np.linspace(Qbar*(1-s),Qbar*s, int((2*s-1)*1000)+1)
    profitlist=[]
    optEset=[]
    for i in range(len(e1list)):
        h1,h2=harvestFunction(a,b,c1,c2,d,e1list[i],e2list[i])
        profitlist.append(totalProfit(a,b,c1,c2,d,h1,h2))
    for x in indexSet(profitlist): 
        optEset.append((e1list[x],e2list[x]))
    return optEset

def cost1(h1,c1,d):
     return c1*h1+0.5*d*h1**2
def cost2(h2,c2,d):
     return c2*h2+0.5*d*h2**2
    
def mincost(Q,c1,c2,d):
    if d>0:
        if Q<(c2-c1)/d:
            return cost1(Q,c1,d)
        else:
            q1=Q/2-(c1-(c1+c2)/2)/d
            q2=Q/2-(c2-(c1+c2)/2)/d
            return cost1(q1,c1,d)+cost2(q2,c2,d)
    elif d==0:
        return cost1(Q,c1,d)

def efficiencyIndex(c1,c2,d,h1,h2):
    return mincost(h1+h2,c1,c2,d)/costFcn(c1,c2,d,h1,h2)

def profit1(a,b,c1,c2,d,h1,h2):
    return (a-b*(h1+h2))*h1 -cost1(h1,c1,d)

def profit2(a,b,c1,c2,d,h1,h2):
    return (a-b*(h1+h2))*h2 -cost2(h2,c2,d)

def zeroProfitQ(a,b,c1,c2,d):
    Qhat=(2*a-c1-c2)/(2*b+d)    
    return Qhat

# 1st best welfare
def maxWelfare(a,b,c1,c2,d,q1,q2):
    Q=min(q1+q2, zeroProfitQ(a,b,c1,c2,d))
    return (a+a-b*Q)*Q/2-  mincost(Q,c1,c2,d)

def realizedWelfareRatio(a,b,c1,c2,d,h1,h2,q1,q2):
    return  welfare(a,b,c1,c2,d,h1,h2)/maxWelfare(a,b,c1,c2,d,q1,q2)

In [3]:
def plotEfficiency_sGraph2(a=10,xlen=5,b=1,c1=3,c2=4,d=1,Qbar=4,s=1,caseName='Efficiency Changing s',ylow=0.8,yhigh=1.02,showCEff=False,saveP=False):
    fig=plt.figure(figsize=(8,6))
    ax1=fig.add_subplot(111)
    
    ax1.set(xlim=[0.5,1],ylim=[ylow,yhigh])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(2)
    ax1.spines['bottom'].set_linewidth(2)
    ax1.yaxis.set_ticks_position('none') 
    ax1.xaxis.set_ticks_position('none') 
    ax1.set_yticklabels([])
    ax1.set_xticklabels([])

    slist=np.linspace(0.5,1,51)
    costEffIdxLst=[]
    realizedRatioLst=[]
    for s in slist:
        e1star,e2star = optE(a,b,c1,c2,d,Qbar,s)[0]  
        h1star,h2star = harvestFunction(a,b,c1,c2,d,e1star,e2star)  
        costEffIdx=round(efficiencyIndex(c1,c2,d,h1star,h2star),4)
        realizedRatio = round(realizedWelfareRatio(a,b,c1,c2,d,h1star,h2star,e1star,e2star),4)
        costEffIdxLst.append(costEffIdx)
        realizedRatioLst.append(realizedRatio)
    
    if showCEff:
        ax1.plot(slist,costEffIdxLst,label='Cost Efficiency Index',marker='o',color="black",linestyle="dashed",markersize=4)
    ax1.plot(slist,realizedRatioLst,label='Realized Welfare Ratio',marker='x',color="black",linestyle="dotted")
    #ax1.plot(slist,realizedRatioLst,label='Realized Welfare Ratio',color="black",lw=2.5)

    ax1.legend()
    strrr="a=" + str(a)+ ", b=" + str(b)+", c1="+ str(c1) + " , c2=" + str(c2) + " , d="+ str(d)+ " , Q="+ str(Qbar)+ "  Efficiency when Changing s"
    fig.suptitle(strrr)
    # Label the parameter 
    #ax1.text(2,4,strrr,color='black',fontsize=13)
    fname=caseName+'.png'
    if saveP:
        plt.savefig(fname,dpi=290)
    plt.show()

In [4]:
def plotChangingS(a=10,xlen=6.7,b=1,c1=3,c2=4,d=1,Qbar=4,s=1,prlv=10):
    fig=plt.figure(figsize=(10,10))
    ax1=fig.add_subplot(111)

    ax1.set(xlim=[0,xlen],ylim=[0,xlen])
    ax1.xaxis.set_ticks_position('none') 
    ax1.yaxis.set_ticks_position('none') 
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(2.5)
    ax1.spines['bottom'].set_linewidth(2.5)
    xlist=np.linspace(0.001,xlen,101)
    ylist=np.linspace(0.001,xlen,101)
    ax1.set_yticklabels([])
    ax1.set_xticklabels([])

    #draw BRs
    br2=[]
    br1=[]
    for i in range(len(xlist)):
        br2.append(BR2(a,b,c2,d,xlist[i]))
        br1.append(BR1(a,b,c1,d,ylist[i]))
    ax1.plot(xlist,br2,label='BRs',color='black',linewidth=1,linestyle='dashed',alpha=0.1)
    ax1.plot(br1,ylist,color='black',linewidth=1,linestyle='dashed',alpha=0.1)
    
    
    X,Y = np.meshgrid(xlist,ylist)
    fProfit= lambda x,y: totalProfit(a,b,c1,c2,d,x,y)
    fWelfare= lambda x,y: welfare(a,b,c1,c2,d,x,y)
    
    ZProfit=fProfit(X,Y)
    ZWelfare=fWelfare(X,Y)
    ZProfit[ZProfit<0]=0
    ZWelfare[ZWelfare<0]=0
    
    
    fzeffic=lambda x,y : efficiencyIndex(c1,c2,d,x,y)
    Zef=np.zeros(np.shape(X))
    for i in range (np.shape(Zef)[0]):
        for j in range (np.shape(Zef)[1]):
            Zef[i,j]=fzeffic(X[i,j],Y[i,j])
    
    cp=ax1.contour(X, Y, ZProfit,levels=prlv ,linewidths=1,linestyles='dashed',colors='black')

    xlist=np.linspace(0.00001,5,51)
    ylist=np.linspace(0.00001,5,51)
    #ax1.plot(xlist,(1-s)/s *xlist, label='Share Cap',color='orange',linewidth=1.3,linestyle='-.',alpha=0.6)
    #ax1.plot((1-s)/s * ylist,ylist,color='orange',linewidth=1.3,linestyle='-.',alpha=0.6)
    h1list=[]
    h2list=[]
    e1list=[]
    e2list=[]

    slist=np.linspace(0.5,1,101)
    for i in range(len(slist)):
        e1,e2 = optE(a,b,c1,c2,d,Qbar,slist[i])[0]
        h1,h2 = harvestFunction(a,b,c1,c2,d,e1,e2)
        e1list.append(e1)
        e2list.append(e2)
        h1list.append(h1)
        h2list.append(h2)
        
    ax1.scatter(e1list,e2list,c=range(len(e1list)),cmap='viridis_r',s=15,alpha=0.7,marker='x')#,label="QB holding movement")
    ax1.scatter(h1list,h2list,c=range(len(e1list)),cmap='viridis_r',s=4,alpha=0.7,marker='o')#,label="Harvest movement")    
    ax1.plot(xlist,xlist,linewidth=0.1,c='black')
    
    for Q in np.linspace(0,5,201):
            qq1=Q/2-(c1-(c1+c2)/2)/d
            qq2=Q/2-(c2-(c1+c2)/2)/d
            ax1.scatter(qq1,qq2,s=2,alpha=0.7,marker='.',color='gray')#,label="Harvest movement")    

    strrr="a=" + str(a)+ ", b=" + str(b)+", c1="+ str(c1) + " , c2=" + str(c2) + " , d="+ str(d)+ " , Q="+ str(Qbar)
    fig.suptitle(strrr)
    plt.show()

In [5]:
def returnBdRegionB1(a,b,c1,c2,d):
    h1_eff= lambda Q: Q/2-(c1-c2)/(2*d)
    h2_eff= lambda Q: Q/2-(c2-c1)/(2*d)
    br2=lambda h1: (a-b*h1-c2)/(2*b+d)

    fcn = lambda Q: totalProfit(a,b,c1,c2,d,h1_eff(Q),h2_eff(Q))-totalProfit(a,b,c1,c2,d,h2_eff(Q),br2(h2_eff(Q)))
    x=fsolve(fcn,a)[0]
    return x

In [6]:
def interactivePlotAll(a,b,c1,c2,d,Qbar):
    plt.close()
    fig=plt.figure(figsize=(9,6.1))
    gs = gridspec.GridSpec(2, 3)
    ax2=fig.add_subplot(gs[:, 1:3])
    ax0=fig.add_subplot(gs[0, 0])
    ax1=fig.add_subplot(gs[1, 0])
    
    
    ax0.set_xlim(c1,8)
    ax0.set_ylim(0,8)
    ax0.set_xlabel(r'$\Delta_\theta$')
    ax0.set_ylabel(r'$\bar{Q}$')
    ax0.set_title("Parameters")
    ax1.set_xlabel('s')
    ax1.set_title(r'$\phi^{W}(s)$ function')
    ax1.set_xlim(0.5,1)
    ax1.set_ylim(0.8,1)
    ax2.set_xlim(0,a*0.75)
    ax2.set_ylim(0,a*0.75)
    ax2.set_title("Corresponding BR Graph")

    xlist = np.linspace(0, 10, 100)
    ylist = np.linspace(0, 10, 100)
    X, Y = np.meshgrid(xlist, ylist)
    Q2hatFn=lambda c2: 1/(2*b+0.5*d) * (a-(c1+c2)/2 +  b/(2*d*(2*b+d)*(b*b+4*b*d+d*d))**0.5  *( (a-c2)*d -2*b*(c2-c1)))

    blist=np.linspace(0.1,2,21)
    dlist=np.linspace(0.1,2,21)
    slist=np.linspace(0.5,1,101)

    c2list=np.linspace(c1,8,51)
    X,Y = np.meshgrid(xlist,ylist)
    P1M=(a*b+a*d+b*c1)/(2*b+d) 
    MC1M= (a*d+2*b*c1)/(2*b+d) 
    Q1M=(a-c1)/(2*b+d)
    C2U=(b+d)/(2*b+d)*P1M+ b/(2*b+d)*MC1M
    Q2hat0= Q2hatFn(c1)
    x = [c1,MC1M,C2U,C2U]
    y = [Q2hat0,Q1M,Q1M,a]
    ax0.plot(x, y)
    ax0.scatter(c2, Qbar,c='r')
    costEffIdxLst=[]
    realizedRatioLst=[]
    h1list=[]
    h2list=[]
    e1list=[]
    e2list=[]
    for s in slist:
        e1star,e2star = optE(a,b,c1,c2,d,Qbar,s)[0]  
        h1star,h2star = harvestFunction(a,b,c1,c2,d,e1star,e2star) 
        e1list.append(e1star)
        e2list.append(e2star)
        h1list.append(h1star)
        h2list.append(h2star)
        costEffIdx=round(efficiencyIndex(c1,c2,d,h1star,h2star),4)
        realizedRatio = round(realizedWelfareRatio(a,b,c1,c2,d,h1star,h2star,e1star,e2star),4)
        costEffIdxLst.append(costEffIdx)
        realizedRatioLst.append(realizedRatio)
    
    #ax1.plot(slist,realizedRatioLst,color="black") #
    ax1.scatter(slist,realizedRatioLst,c=range(len(e1list)),cmap='viridis_r',s=4,alpha=0.7,marker='o')
    fProfit= lambda x,y: totalProfit(a,b,c1,c2,d,x,y)    
    ZProfit=fProfit(X,Y)
    ZProfit[ZProfit<0]=0
    ax2.contour(X, Y, ZProfit,levels=8 ,linewidths=1,linestyles='dashed',colors='black',alpha=0.5) #
    ax2.scatter(e1list,e2list,c=range(len(e1list)),cmap='viridis_r',s=15,alpha=0.7,marker='x')#,label="QB holding movement")
    ax2.scatter(h1list,h2list,c=range(len(e1list)),cmap='viridis_r',s=4,alpha=0.7,marker='o')#,label="Harvest movement") 
    ax2.plot(np.linspace(0,5,10),np.linspace(0,5,10),linewidth=0.1,c='black')
    
    for Q in np.linspace(0,5,41):
            qq1=Q/2-(c1-(c1+c2)/2)/d
            qq2=Q/2-(c2-(c1+c2)/2)/d
            ax2.scatter(qq1,qq2,s=2,alpha=0.7,marker='.',color='gray')#,label="Harvest movement")    
     
    br2=[]
    br1=[]
    for j in range(len(xlist)):
        br2.append(BR2(a,b,c2,d,xlist[j]))
        br1.append(BR1(a,b,c1,d,xlist[j]))
    ax2.plot(xlist,br2,color='black',linewidth=1,linestyle='dashed',alpha=0.1)
    ax2.plot(br1,xlist,color='black',linewidth=1,linestyle='dashed',alpha=0.1)
    Qe= lambda c2: (2*a-c1-c2+b*(c1-c2)/d )/(3*b+d)  

    QeNew= lambda c2: 1/(2*b+0.5*d) * (a-(c2+c1)/2 +  b/(2*d*(2*b+d)*(b*b+4*b*d+d*d))**0.5  *( (a-c1)*d -2*b*(c1-c2)))
    xlist1=np.linspace(c1,MC1M,100)
    ax0.plot(xlist1,Qe(xlist1),lw=1,alpha=0.5)
    ax0.plot(xlist1,QeNew(xlist1),lw=1,alpha=0.5)
    bdr1=[]
    for c2 in xlist1:
        bdr1.append(returnBdRegionB1(a,b,c1,c2,d))
    ax0.plot(xlist1,bdr1,lw=1,alpha=0.5)    
    strrr=r"$\alpha=$" + str(a)+ r", $\beta=$" + str(round(b,2))+r"$, \theta_1=$"+ str(c1) + "\n "+ r"$, \eta=$"+ str(round(d,2))+ r"$, \bar{Q}=$"+ str(round(Qbar,2))
    ax0.annotate(strrr,(1,1))
    plt.tight_layout()
    plt.show()

In [7]:
interact (interactivePlotAll,a= fixed(10),b=(0.1,3,0.1),c1=fixed(0),c2=(0,8,0.1),d=(0.1,3,0.1),Qbar=(1,10,0.1))

interactive(children=(FloatSlider(value=1.5000000000000002, description='b', max=3.0, min=0.1), FloatSlider(va…

<function __main__.interactivePlotAll>

In [8]:
nbi.publish("XinyuGuo17/interactivePlot/master", "Interactive to HTML.ipynb")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'Interactive to HTML.ipynb'.
Converting notebook...



Successfully converted!

<a href="Interactive to HTML.html" target="_blank" download>Click to download your webpage.</a>

To host your webpage, see the documentation:
<a href="https://www.nbinteract.com/tutorial/tutorial_publishing.html"
        target="_blank">
    https://www.nbinteract.com/tutorial/tutorial_publishing.html
</a>
